In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [0]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 1225377879585711517, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 14843097310175648441
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 4551530376532886185
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15701401920
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 643053043211529335
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]

In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import *
import seaborn as sns

sns.set()
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter

import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

print(torch.__version__)

path = '/content/drive/My Drive/DeepGQuad/'
os.chdir(path)
os.listdir()

1.4.0


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/DeepGQuad/'

In [0]:
!nvidia-smi

Tue May 26 13:21:01 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
columns = ['chr','start','end','seq']
df = pd.read_csv(f'./Data/chr1.bed',sep='\t',header=None)
df.columns = columns
df.head(5)

,chr,start,end,seq
0,chr1,713977,714310,ACCATGGCGCCCCAGTGATGTAGCCGAACACCCGCGCCTCTAACGT...
1,chr1,762850,762982,GGAGGGCACTCACCCGAGCGGACCTTGGCTCCGGATAATCCGTTTC...
2,chr1,840076,840205,gccgcctccgaacgtggccgccgcctcctccgaacgtggccgcttc...
3,chr1,894635,894796,CGTGCACCCCACTTCCGGCCCCAGAATGCCGCGCGGCTGCGCACTT...
4,chr1,935414,935699,GCGGGCGAGCGGCGAGCGCGCGGCGATCCGAGCCCCTAGGGCGGAT...


In [0]:
# Load first chromosome data
with open('Data/chr1.fa') as f:
  seq = f.read()
seq = seq[6:]
seq = seq.replace('\n','')
seq = seq.lower()

In [0]:
# Define sequnce regione
print(len(seq))
start = 700000#10000
end = 3700000
s = seq[start:end]

249250621

In [0]:
# Tokenize nucleotids
s_encoded = np.zeros([len(s),3],dtype=np.int8)
dna_dict = {'a':0, 't':1,'c':2, 'g':3, 'n':np.random.randint(0,4)}
for i, nucl in enumerate(s):
  if i % 1000000 ==0:
    print(i)
  s_encoded[i,0] = dna_dict[nucl]

0


In [0]:
# Label data
count = 0
for i, row in df.iterrows():
  if row.chr == 'chr1':
    if start < row.start and row.end < end:
      count+=1
      s_encoded[row.start:row.end,1:] = [1,1]
count

2

In [0]:
np.savez_compressed('./Data/g4_quad/g4quad_1.npz',s_encoded)

In [0]:
np.savez_compressed('./Data/g4_quad/g4quad_small.npz',s_encoded)

In [0]:
s_encoded = np.load('./Data/g4_quad/g4quad_1.npz',allow_pickle=True)

In [0]:
#!python scripts/eval_transformer_xl.py --model_path output/-g4quad_1.npz/20200526-113720/model.pt

In [0]:
#!chmod +x scripts/run_trasformer_xl.sh

In [0]:
!./scripts/run_trasformer_xl.sh

Run training...
data/g4_quad/g4quad_1.npz
Namespace(adaptive=False, batch_chunk=1, batch_size=10, clamp_len=-1, clip=0.25, clip_nonemb=False, conv_emb=False, conv_size=7, cuda=True, d_embed=32, d_head=32, d_inner=256, d_model=32, data_path='data/g4_quad/g4quad_1.npz', debug=False, decay_rate=0.5, dropatt=0.0, dropout=0.1, emb_init='normal', emb_init_range=0.01, emb_path=None, eta_min=0.0, eval_interval=3000, eval_tgt_len=512, ext_ds=0, gpu0_bsz=0, init='normal', init_range=0.1, init_std=0.02, log_interval=200, lr=0.0001, lr_min=0.0, max_eval_steps=-1, max_step=85000, mem_len=0, methylation=False, multi_gpu=False, n_head=6, n_layer=6, not_tied=True, optim='adam', optim_type=0, output_path='output/', patience=0, pre_conv=False, proj_init_std=0.01, restart=False, restart_dir='', same_length=False, scheduler='cosine', seed=2222, shift=20, tgt_len=512, varlen=True, warmup_step=3000)
Full dataset size: 990000
Train set size: 1386000
Test set size: 396000
building vocab with min_freq=0, max_s